In [323]:
import pandas as pd
import numpy as np
import math

In [324]:
def obtainRAandDECCoords(matrixRA, matrixDEC, x, y):
  raAndDec = np.array([matrixRA[0], matrixDEC[0]]) + np.array([[matrixRA[1], matrixRA[2]], [matrixDEC[1], matrixDEC[2]]]) @ np.array([x, y])
  xRA = raAndDec[0] / 15
  yDec = raAndDec[1]  

  print(xRA, yDec)

  xString = str(int(xRA)) + ":" + str(int((xRA % 1) * 60)) + ":" + str(round(((((xRA % 1) * 60) % 1) * 60), 2))
  yString = "+" + str(int(yDec)) + ":" + str(int((yDec % 1) * 60)) + ":" + str(round(((((yDec % 1) * 60) % 1) * 60), 2))

  return xString, yString

In [325]:
def obtainData(inputFile):
  data = pd.read_csv(inputFile, sep = " ", names = ['X Centroid', 'Y Centroid', 'RA', 'DEC'])
  return data

In [326]:
def obtainInvMatrixA(data):
  N  = len(data)
  sumX = data['X Centroid'].sum()
  sumY = data['Y Centroid'].sum()
  xCentroid = data['X Centroid'].to_numpy()
  yCentroid = data['Y Centroid'].to_numpy()

  sumXSqr = 0
  sumYSqr = 0
  sumXtY = 0

  for i in range(len(xCentroid)):
    sumXSqr += xCentroid[i] ** 2
    sumYSqr += yCentroid[i] ** 2
    sumXtY += xCentroid[i] * yCentroid[i]
  
  matrixA = np.array([[N, sumX, sumY],
       [sumX, sumXSqr, sumXtY],
       [sumY, sumXtY, sumYSqr]])
  
  return np.linalg.inv(matrixA)

In [327]:
def obtainRAandDecMatrix(data):
  raArray = data['RA'].to_numpy()
  decArray = data['DEC'].to_numpy()
  xCentroid = data['X Centroid'].to_numpy()
  yCentroid = data['Y Centroid'].to_numpy()

  raArray = raToDecimal(raArray)
  decArray = decToDecimal(decArray)

  #Sum Variables
  sumRaTX = 0 #Sum of RA values times X Values
  sumRaTY = 0 #Sum of RA values times Y Values

  sumDecTX = 0 #Sum of DEC values times X Values
  sumDecTY = 0 #Sum of DEC values times Y Values

  for i in range(len(xCentroid)):
    sumRaTX += raArray[i] * xCentroid[i]
    sumRaTY += raArray[i] * yCentroid[i]

    sumDecTX += decArray[i] * xCentroid[i]
    sumDecTY += decArray[i] * yCentroid[i]

  raMatrix = [sum(raArray), sumRaTX, sumRaTY]
  decMatrix = [sum(decArray), sumDecTX, sumDecTY]

  return raMatrix, decMatrix, raArray, decArray

In [328]:
def raToDecimal(data):
  decimalRA = []
  for i in range(len(data)):
    tempValues = np.array(data[i].split(":")).astype(float)
    tempDecimal = (tempValues[0] + (tempValues[1]/60) + (tempValues[2]/3600)) * 15
    decimalRA.append(tempDecimal)
  return decimalRA

def decToDecimal(data):
  decimalDEC = []
  for i in range(len(data)):
    tempString = data[i].replace("+", "")
    tempValues = np.array(tempString.split(":")).astype(float)
    tempDecimal = tempValues[0] + (tempValues[1]/60) + (tempValues[2]/3600)
    decimalDEC.append(tempDecimal)
  return decimalDEC

In [329]:
def obtainUncertainty(matrixRA, matrixDec, raArray, decArray, data):
  N = len(decArray)

  xCentroid = data['X Centroid'].to_numpy()
  yCentroid = data['Y Centroid'].to_numpy()

  uncertaintyRANumerator = 0
  uncertainityDECNumerator = 0

  for i in range(len(raArray)):

    uncertaintyRANumerator += (raArray[i] - matrixRA[0] - (matrixRA[1] * xCentroid[i]) - (matrixRA[2] * yCentroid[i])) ** 2

    uncertainityDECNumerator += (decArray[i] - matrixDec[0] - (matrixDec[1] * xCentroid[i]) - (matrixDec[2] * yCentroid[i])) ** 2
  
  uncertaintyRA = math.sqrt(uncertaintyRANumerator / (N - 3)) * 3600
  uncertaintyDEC = math.sqrt(uncertainityDECNumerator / (N - 3)) * 3600

  return uncertaintyRA, uncertaintyDEC

In [330]:
def LSPR(inputFile):

  x = float(input("X Coordinate: "))
  y = float(input("Y Coordinate: "))

  starData = obtainData(inputFile)
  invMatrixA = obtainInvMatrixA(starData)
  print(invMatrixA)
  raMatrix, decMatrix, raArray, decArray = obtainRAandDecMatrix(starData)

  matrixOne = np.matmul(raMatrix, invMatrixA)
  matrixTwo = np.matmul(decMatrix, invMatrixA)

  uncertaintyRA, uncertaintyDEC = obtainUncertainty(matrixOne, matrixTwo, raArray, decArray, starData)

  raFinal, decFinal = obtainRAandDECCoords(matrixOne, matrixTwo, x, y)

  # Data Printing
  print("""***************
plate constants
***************
""")

  print("b1: " + str(matrixOne[0]) + " deg")
  print("b2: " + str(matrixTwo[0]) + " deg")
  print("a11: " + str(matrixOne[1]) + " deg/pix")
  print("a12: " + str(matrixOne[2]) + " deg/pix")
  print("a21: " + str(matrixTwo[1]) + " deg/pix")
  print("a22: " + str(matrixTwo[2]) + " deg/pix")


  print("""***********
uncertainty
***********
""")

  print("RA: " + str(uncertaintyRA) + " arcsec")
  print("Dec: " + str(uncertaintyDEC) + " arcsec")


  print("""
*********************************
astrometry for (x,y) = (""" + str(x) +", " + str(y) + ")" + """
*********************************
""")
  
  print("RA = " + raFinal)
  print("Dec = " + decFinal)

In [331]:
LSPR('/content/LSPRtestinput1.txt')

X Coordinate: 484.35
Y Coordinate: 382.62
[[ 5.86018891e+00 -5.58178506e-03 -8.34577581e-03]
 [-5.58178506e-03  1.01685826e-05  1.95734623e-06]
 [-8.34577581e-03  1.95734623e-06  1.98661891e-05]]
19.17293383862371 35.635590205001826
***************
plate constants
***************

b1: 287.81234897646937 deg
b2: 35.7670276378642 deg
a11: -0.0004402433954684426 deg/pix
a12: -1.335400271429421e-05 deg/pix
a21: 1.1056625049132851e-05 deg/pix
a22: -0.00035751583609044957 deg/pix
***********
uncertainty
***********

RA: 0.06950232651518844 arcsec
Dec: 0.05336088932401711 arcsec

*********************************
astrometry for (x,y) = (484.35, 382.62)
*********************************

RA = 19:10:22.56
Dec = +35:38:8.12
